# Bad Apple Storyboard
This is a notebook to convert bad apple video to osu! storyboard, in order to do it, you need the following requirements:
- Python 3.8 (Due to typing)
- Ffmpeg
- osbpy
- Pillow
- tqdm

## Install Dependencies
To install them, run the following command.

In [1]:
# Please install ffmpeg on your own.
# https://ffmpeg.org/download.html
!pip install osbpy pillow tqdm

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


# Extract Frames

Next, we need to extract frames from the video. (Please edit `badapple.mp4` with your video path, and `30` with `fps`)

In [2]:
import os
os.makedirs("frames", exist_ok=True)
!ffmpeg -i badapple.mp4 -r 30 frames/%03d.jpg

ffmpeg version 4.2.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.1 (GCC) 20200523
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt
  libavutil      56. 31.100 / 56. 31.100
  li

# Set Generator Method
Before we go, we need to set up which method should we use to generate the storyboard. Currently, there are two options available:
- Quadtree: Basically how it works is making each frame a chunk of 4 and repeat it. This is recommended because the result would have less particles being rendered at once.
- Pixels: Think of it as if how monitors work, you light up each pixels.

The default is to use Quadtree.

In [4]:
USE_PIXELS=False
if USE_PIXELS:
    from pixels import osb, pixel_extract
else:
    from quadtree import osb, pixel_extract

# Extract Alpha Data
Up next, we need to extract all alpha data from every single frame from the video.
By default, it splits to 16 chunks of data and running in 2 threads. If you have big memory, then raise the number of threads and reduce the number of splits, and vice-versa.

Please take note that you will need to have at least 6GB of RAM for default config!

In this part you will also need to decide how big each square is. Smaller quality means it is more high resolution but more resource exhaustive. However, there is a note:
- Quadtree: Your quality needs to be between 0-6.
- Pixels: Your quality needs to be a factor of 640 and 480

The default for this is 0 for quadtree, 10

In [5]:
if USE_PIXELS:
    quality = 10
else:
    quality = 0
output_filename = "thing.osb"

In [6]:
pixel_extract.run(
    quality,
    number_of_thread=2,
    number_of_splits=16
)

TypeError: run() got multiple values for argument 'number_of_thread'

## Generate osb

With all of the alpha datas that is available, we can now generate the osb.
Please put in the correct fps of the frames you were extracting and the transparency precision, and adjust the music offset.

Higher transparency precision means that you will have higher file size and performance bottlenecks.

In [ ]:
osb.generate_osb(
    quality,
    output_filename,
    fps=30,
    transparency_precision=4,
    music_offset=0,
)